In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
### Import Libraries

# Azure Data Lake libraries
import common.utils.azure_data_lake_interface as adl

# Helper function libraries
from common.utils.configuration_management import load_config

# Data analysis libraries
import pandas as pd

### Step 1: Update the raw data via latest JSONs

### Step 2: Run the prepare_transactions script to clean/enhance raw data

### Step 3: Get Item Master and PO Line Item Data from Data Lake

In [5]:
# attach to the data lake
config = load_config("common/config/datalake_config.json")
service_client = adl.get_azure_service_client(config["blob_url"])
file_system_client = adl.get_azure_file_system_client(service_client, "consolidated")

# get data
data_state = "enhanced"
trans_type = "PurchOrd"

item_master_df = adl.get_parquet_file_from_data_lake(file_system_client, f"{data_state}/netsuite", f"items_{data_state}.parquet")

filename = f"transaction/{trans_type}ItemLineItems_{data_state}.parquet"
po_lines_df = adl.get_parquet_file_from_data_lake(file_system_client, f"{data_state}/netsuite", filename)

### Step 4: Get the transactions of interest

In [6]:
trans_type = "Estimate"
filename = f"transaction/{trans_type}ItemLineItems_{data_state}.parquet"
est_lines_df = adl.get_parquet_file_from_data_lake(file_system_client, f"{data_state}/netsuite", filename)
saved_est_lines_df = est_lines_df.copy()

,amount,assembly_component,cost_estimate_type,created_from,custcol_ava_taxamount,custcol_sa_quote_po_rate,custom_manufacturer,customer_id,description,display_name,...,level_3_category,level_4_category,level_5_category,level_6_category,total_amount,total_cost,gross_profit,gross_profit_percent,highest_recent_cost,highest_quoted_cost
0,50340.0,False,CUSTOM,None,0.00,0.0,None,9149,None,None,...,None,None,None,None,50340.0,40272.00,10068.00,0.200000,NaN,20136.00
1,3025.1,False,CUSTOM,None,0.00,0.0,CDC,53020,"HPX-95 1""",Gas/Liquid Rupture Disc,...,Not Specified,Not Specified,Not Specified,Not Specified,3025.1,2186.40,838.70,0.277247,NaN,2186.40
2,3773.7,False,CUSTOM,None,0.00,0.0,CDC,53020,"HPX-95 1""",Gas/Liquid Rupture Disc,...,Not Specified,Not Specified,Not Specified,Not Specified,3773.7,2709.60,1064.10,0.281978,NaN,2186.40
3,4317.0,False,CUSTOM,None,0.00,0.0,Consolidated,21390,"1910EC-2\r\n1"" CONSOLIDATED SRV",1x2 (300#x150#),...,PRV,1900,1,New,4317.0,2857.44,1459.56,0.338096,2316.48,2857.44
4,940.0,False,CUSTOM,None,0.00,0.0,None,218144,"1"" Safety Valve Repair",None,...,None,None,None,None,940.0,246.88,693.12,0.737362,NaN,246.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407298,14466.0,False,PURCHPRICE,None,0.00,0.0,General Twin Seal,9560,"8"" 300# LOW TEMP MODEL 8821LT-751G GEAR OPERAT...",None,...,None,None,None,None,14466.0,10849.00,3617.00,0.250035,NaN,10849.00
407299,3571.0,False,CUSTOM,None,178.55,0.0,None,29561,None,None,...,None,None,None,None,3571.0,2500.00,1071.00,0.299916,NaN,334914.00
407300,358.0,False,CUSTOM,None,17.90,0.0,None,29561,None,None,...,None,None,None,None,358.0,250.00,108.00,0.301676,NaN,334914.00
407301,2240.0,False,ITEMDEFINED,None,0.00,0.0,None,29561,Hours Standard SV/Line Valve Technician Time,None,...,None,None,None,None,2240.0,592.00,1648.00,0.735714,NaN,37.00
